Part 2: Descriptive Statistics



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("results.csv")
filtered = df[df["eGFR"] <= 65].copy()
filtered[["age", "height", "weight"]].describe()


In [ ]:
# Load demographics and show descriptive statistics for age, height, and weight
import pandas as pd

dem_url = "https://ruby.ils.unc.edu/~ahhardin/chip690_335_2025_fall/patient_demographics.csv"
df = pd.read_csv(dem_url)

# Ensure numeric types
for col in ["age", "height_inches", "weight_lbs"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Compute descriptive statistics and IQR
stats = df[["age", "height_inches", "weight_lbs"]].describe().T
stats["IQR"] = iqr

# Display

print("Descriptive statistics for Age, Height (inches), and Weight (lbs):")
display(stats.style.format({
    "count": "{:.0f}",
    "mean": "{:.2f}",
    "std": "{:.2f}",
    "min": "{:.2f}",
    "25%": "{:.2f}",
    "50%": "{:.2f}",
    "75%": "{:.2f}",
    "max": "{:.2f}",
    "IQR": "{:.2f}"
}))
